<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Практический анализ данных и машинное обучение"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: Лисицын Сергей
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Занятие 9. Продвинутые методы классификации и регрессии. Разреженные данные</center>

## <center>Часть 2. Vowpal Wabbit</center>

Vowpal Wabbit (VW) является одной из наиболее широкоиспользуемых библиотек в индустрии. Её отличает высокая скорость работы и поддержка большого количества различных режимов обучения. Особый интерес для больших и высокоразмерных данных представляет онлайн-обучение - самая сильная сторона библиотеки. 


Основным интерфейсом для работы с VW является shell

In [18]:
import sklearn.datasets
import re

Для того, чтобы изучить возможные режимы работы vw воспользуемся командой ! для запуска:

In [19]:
!vw --help

Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = 
num sources = 1


VW options:
  --random_seed arg                     seed random number generator
  --ring_size arg                       size of example ring

Update options:
  -l [ --learning_rate ] arg            Set learning rate
  --power_t arg                         t power value
  --decay_learning_rate arg             Set Decay factor for learning_rate 
                                        between passes
  --initial_t arg                       initial t value
  --feature_mask arg                    Use existing regressor to determine 
                                        which parameters may be updated.  If no
                                        initial_regressor given, also used for 
                                        initial weights.

Weight options:
  -i [ --initial_regressor ] arg        Initial regressor(s)
  --initial_weight arg                  Set all 

Vowpal Wabbit считывает данные из файла или стандартного ввода (stdin) в формате, который имеет следующий вид:

`[Label] [Importance] [Tag]|Namespace Features |Namespace Features ... |Namespace Features`

`Namespace=String[:Value]`

`Features=(String[:Value] )*`

где [] обозначает необязательные элементы, а (...)\* означает повтор неопределенное число раз. 

- **Label** является числом, "правильным" ответом. В случае классификации обычно принимает значение 1/-1, а в случае регрессии некоторое вещественное число
- **Importance** является числом и отвечает за вес примера при обучении. Это позволяет бороться с проблемой несбалансированных данных, изученной нами ранее
- **Tag** является некоторой строкой без пробелов и отвечает за некоторое "название" примера, которое сохраняется при предсказании ответа. Для того, чтобы отделить Tag от Importance лучше начинать Tag с символа '.
- **Namespace** служит для создания отдельных пространств признаков. В аргументах Namespace именуются по первой букве, это нужно учитывать при выборе их названий
- **Features** являются непосредственно признаками объекта внутри **Namespace**. Признаки по умолчанию имеют вес 1.0, но его можно переопределить, к примеру feature:0.1. 


К примеру, под такой формат подходит следующая строка:

```
1 1.0 |Subject WHAT car is this |Organization University of Maryland:0.5 College Park
```


чтобы убедиться в этом, запустим vw с этим обучающим примером:

In [20]:
! echo '1 1.0 |Subject WHAT car is this |Organization University of Maryland:0.5 College Park' | vw

For more information use: vw --help
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = 
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0   1.0000   0.0000       10

finished run
number of examples per pass = 1
passes used = 1
weighted example sum = 1.000000
weighted label sum = 1.000000
average loss = 1.000000
best constant = 1.000000
best constant's loss = 0.000000
total feature number = 10


VW является прекрасным инструментом для работы с текстовыми данными. Давайте убедимся в этом с помощью выборки 20newsgroups, содержащей письма из 20 различных тематических рассылок:

In [50]:
newsgroups = sklearn.datasets.fetch_20newsgroups('../../data/')

Рассмотрим первый текстовый документ этой коллекции:

In [51]:
text = newsgroups['data'][0]
target = newsgroups['target_names'][newsgroups['target'][0]]

print '-----'
print target
print '-----'
print text.strip()
print '----'

-----
rec.autos
-----
From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----
----


Выполним простейшую обработку текста: избавимся от регистра и найдем все слова длиннее трех символов:

In [52]:
def vowpalize(document, label):
    return str(label) + ' |text ' + ' '.join(re.findall('\w{3,}', text.lower())) + '\n'

vowpalize(text, 1 if target == 'rec.autos' else -1)

u'1 |text from lerxst wam umd edu where thing subject what car this nntp posting host rac3 wam umd edu organization university maryland college park lines was wondering anyone out there could enlighten this car saw the other day was door sports car looked from the late 60s early 70s was called bricklin the doors were really small addition the front bumper was separate from the rest the body this all know anyone can tellme model name engine specs years production where this car made history whatever info you have this funky looking car please mail thanks brought you your neighborhood lerxst\n'

Запишем в файл преобразованные таким образом документы. Будем считать документ положительным, если он относится к рассылке 'rec.autos'. Так мы построим модель, отличающую письма про автомобили от остальных: 

In [55]:
with open('../../data/20news.vw', 'w') as vw_data:
    for text, target in zip(newsgroups['data'], newsgroups['target']):
        vw_data.write(vowpalize(text, 1 if newsgroups['target_names'][target] == 'rec.autos' else -1))

Запустим vw на сформированном файле. С

In [54]:
!vw -d ../../data/20news.vw --loss_function hinge -f ../../data/20news_model.vw

final_regressor = ../../data/20news_model.vw
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = ../../data/20news.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0   1.0000   0.0000      101
1.201162 1.402325            2            2.0  -1.0000   0.4023      117
1.081197 0.961232            4            4.0  -1.0000  -0.1444      109
0.693897 0.306596            8            8.0  -1.0000  -1.0000      354
0.441401 0.188906           16           16.0  -1.0000  -1.0000      149
0.413001 0.384601           32           32.0  -1.0000  -0.8999      186
0.309969 0.206937           64           64.0  -1.0000  -0.5848       47
0.286468 0.262966          128          128.0  -1.0000  -1.0000      340
0.205548 0.124629          256          256.0  -1.0000  -1.0000      230
0.156957 0.1

TODO: анализ результатов